In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from src.benchmarkers import *
from src.benchmarkersV2 import *
import time
from tqdm import tqdm
from matplotlib import pyplot as plt

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

np.random.seed(0)

In [2]:
def print_name(name):
    print(name)

#with h5py.File(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\32\ds_10_core.h5", "r") as file:
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\235\ds_100.parquet")
#with pa.memory_map(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\outputs\v2\235\ds_100_stream.arrows", 'rb') as source:
#        table = pa.ipc.open_stream(source).read_all()
            

In [3]:
#N = [10,100,200,300,500]
N = [10]
#dimensions = [32,64,125,192]
dimensions = [235]
selected_label = 10
iterations = 10

### Generation

In [33]:
#for item in tqdm(N):
#    for dim in dimensions:
#        generator.create_dataset(item,f"outputs/v2/{dim}/ds_{item}",dim)
#        generator.create_arrow_file(f"outputs/v2/{dim}/ds_{item}")
#        generator.create_arrow_stream(f"outputs/v2/{dim}/ds_{item}")

### Loader a dim 

In [ ]:
import os
root_path = ""

classes = sorted(entry.name for entry in os.scandir(root_path) if entry.is_dir())

if not classes:
    raise FileNotFoundError(f"Couldn't find any class folder in {root_path}.")

class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

directory = os.path.expanduser(root_path)

files = []

for target_class in sorted(class_to_idx.keys()):
    class_index = class_to_idx[target_class]
    target_dir = os.path.join(directory, target_class)
    if not os.path.isdir(target_dir):
        continue
    for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
        for fname in sorted(fnames):
            path = os.path.join(root, fname)
            Y = {
                "label": class_index
            }
            X = {
                "path": path
            }
            #Open the image
            #Save the image in the sample structure of our framework
        

### Benchmark Pytorch dataset

In [ ]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

### Inferenza con salvataggio dei tempi (classificazione)